 <a name='ind'/>

# <ins>Indice</ins>:

- ## [Extracción + Transformación](#ext)
- ## [Carga](#load)

In [1]:
import requests as rq
import pandas as pd
import pyarrow
from tqdm import tqdm
import time
import pymongo
from Spotify_id import *
pd.set_option('display.max_columns', None)

<a name='ext'/>

###### ⬆️ [Indice](#ind)

### Extracción + Transformación
***

A partir de la tabla de grupos obtenido en el [proyecto](https://github.com/gusavato/3_IRONHACK_ETL) anterior de ETL. Y generaremos una tabla nueva con el top 10 de canciones de cada artista, obteniendo dicha información de la API de Spotify

In [2]:
grupos = pd.read_parquet('../data/grupos_spotify_clean.parquet')
grupos.head()

,Grupos,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,Del Water Gap,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 122777}","[deep new americana, indie pop, pov: indie]",https://api.spotify.com/v1/artists/0xPoVNPnxII...,0xPoVNPnxIIUS1vrxAYV00,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Del Water Gap,58.0,artist,spotify:artist:0xPoVNPnxIIUS1vrxAYV00
1,León Benavente,{'spotify': 'https://open.spotify.com/artist/5...,"{'href': None, 'total': 74409}",[spanish indie pop],https://api.spotify.com/v1/artists/5wuBA4TiRM0...,5wuBA4TiRM0CNjSXjsutn6,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",León Benavente,42.0,artist,spotify:artist:5wuBA4TiRM0CNjSXjsutn6
2,ATB,{'spotify': 'https://open.spotify.com/artist/7...,"{'href': None, 'total': 354818}","[german dance, german techno, german trance, t...",https://api.spotify.com/v1/artists/7jZM5w05mGh...,7jZM5w05mGhw6wTB1okhD9,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",ATB,64.0,artist,spotify:artist:7jZM5w05mGhw6wTB1okhD9
3,Tyler Joe Miller,{'spotify': 'https://open.spotify.com/artist/1...,"{'href': None, 'total': 4670}",[canadian contemporary country],https://api.spotify.com/v1/artists/1MmpCgmJymS...,1MmpCgmJymS8Etwm9RxuxM,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Tyler Joe Miller,41.0,artist,spotify:artist:1MmpCgmJymS8Etwm9RxuxM
4,Loverground,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 4151}",[],https://api.spotify.com/v1/artists/3SvoerawAn5...,3SvoerawAn5RAZ2N9osc3z,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Loverground,38.0,artist,spotify:artist:3SvoerawAn5RAZ2N9osc3z


In [3]:
AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

In [4]:
# constructaor de url's para top 10 canciones artista

urls = [f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks?market=ES' for artist_id in grupos.id]
artist_url = [{'artist':k, 'url':v} for k,v in zip(grupos.Grupos,urls)]
artist_url[:5]

[{'artist': 'Del Water Gap',
  'url': 'https://api.spotify.com/v1/artists/0xPoVNPnxIIUS1vrxAYV00/top-tracks?market=ES'},
 {'artist': 'León Benavente',
  'url': 'https://api.spotify.com/v1/artists/5wuBA4TiRM0CNjSXjsutn6/top-tracks?market=ES'},
 {'artist': 'ATB',
  'url': 'https://api.spotify.com/v1/artists/7jZM5w05mGhw6wTB1okhD9/top-tracks?market=ES'},
 {'artist': 'Tyler Joe Miller',
  'url': 'https://api.spotify.com/v1/artists/1MmpCgmJymS8Etwm9RxuxM/top-tracks?market=ES'},
 {'artist': 'Loverground',
  'url': 'https://api.spotify.com/v1/artists/3SvoerawAn5RAZ2N9osc3z/top-tracks?market=ES'}]

Creamos una función para obtener la información que deseamos de la API

In [ ]:
def get_top_10_tracks(art_url, headers):  # recibe diccionario key:artista

    # Pausamos el tiempo entre consultas. Evitamos que nos baneen
    time.sleep(0.20)
    temp_lst = []
    try:
        response = rq.get(art_url['url'], headers=headers)
    except:
        temp_dict['artist'] = None
        temp_dict['name'] = None
        temp_dict['track_id'] = None
        temp_dict['popularity'] = None
        temp_dict['href'] = None
        temp_dict['album '] = None
        temp_dict['album_id'] = None
        temp_dict['album_href'] = None
        temp_dict['release_date'] = None

    tracks = response.json()['tracks']  # Lista con el top 10 de tracks

    for track in tracks:
        temp_dict = dict()
        temp_dict['artist'] = art_url['artist']
        temp_dict['name'] = track['name']
        temp_dict['track_id'] = track['id']
        temp_dict['popularity'] = track['popularity']
        temp_dict['href'] = track['href']
        temp_dict['album '] = track['album']['name']
        temp_dict['album_id'] = track['album']['id']
        temp_dict['album_href'] = track['album']['href']
        temp_dict['release_date'] = track['album']['release_date']
        temp_lst.append(temp_dict)
    return temp_lst

In [7]:
lst_tracks_info = [get_top_10_tracks(art_url,headers) for art_url in tqdm(artist_url)]

100%|██████████████████████████████████████████████████████████████████████████████| 4615/4615 [35:40<00:00,  2.16it/s]


In [10]:
lst_tracks_info_flat = []
for lst in lst_tracks_info:
    for track in lst:
        lst_tracks_info_flat.append(track)

In [13]:
tracks = pd.DataFrame(lst_tracks_info_flat)

In [152]:
tracks.head()

,artist,name,track_id,popularity,href,album,album_id,album_href,release_date
0,Del Water Gap,Ode to a Conversation Stuck in Your Throat,6JwkAGwClwUzcOi0ZBbSLe,61,https://api.spotify.com/v1/tracks/6JwkAGwClwUz...,Ode to a Conversation Stuck in Your Throat,5e8S6KvNbnZBbNzaL2YcLJ,https://api.spotify.com/v1/albums/5e8S6KvNbnZB...,2020-05-01
1,Del Water Gap,Better Than I Know Myself,4PawewnYu8lyLk3DX1RCNu,59,https://api.spotify.com/v1/tracks/4PawewnYu8ly...,Del Water Gap,22ljnmjYzy4TS5tCtaRIUE,https://api.spotify.com/v1/albums/22ljnmjYzy4T...,2021-10-08
2,Del Water Gap,New Song,3PUMPtOSeXSJsBvK43K96b,59,https://api.spotify.com/v1/tracks/3PUMPtOSeXSJ...,Notes from the Archive: Recordings 2011-2016,2li1FMT6riytgLdq4Eu4rb,https://api.spotify.com/v1/albums/2li1FMT6riyt...,2020-12-18
3,Del Water Gap,Perfume,3W3GKw4by2pdPfG8bYQMVG,57,https://api.spotify.com/v1/tracks/3W3GKw4by2pd...,Del Water Gap,22ljnmjYzy4TS5tCtaRIUE,https://api.spotify.com/v1/albums/22ljnmjYzy4T...,2021-10-08
4,Del Water Gap,High Tops,1VZQYv1EhvZAwKYYdgmcdf,57,https://api.spotify.com/v1/tracks/1VZQYv1EhvZA...,High Tops,4NMOX3QV2My3pNgg0hu8hX,https://api.spotify.com/v1/albums/4NMOX3QV2My3...,2018-10-11


El tiempo de extracción ha sido elevado (30 minutos) por lo que son buenas prácticas salvar la información obtenida para prevenir posibles perdidas de datos

In [14]:
tracks.to_parquet('../data/tracks_raw.parquet',engine='pyarrow')

A continuación realizaremos otro proceso de extracción que nos permita obtener la información de cada una de las canciones

In [154]:
def get_audio_feautures(lst_tracks, headers):  # recibe diccionario key:artista

    # Pausamos el tiempo entre consultas. Evitamos que nos baneen
    time.sleep(0.11)
    lst_id = '%2C'.join(lst_tracks)
    url = f'https://api.spotify.com/v1/audio-features?ids={lst_id}'

    response = rq.get(url, headers=headers)
    return response.json()

La API de Spotify permite consultar en una consulta hasta 100 canciones, por lo que segmentaremos nuestra lista de más de 40000 canciones en 433 listas de 100 canciones, y lanzaremos consulta por cada lista

In [155]:
lst_afeautures = []
for i in tqdm(range(len(tracks.track_id)//100)):
    lst = tracks.track_id[i*100:(i+1)*100]
    lst_afeautures.append(get_audio_feautures(lst,headers))

100%|████████████████████████████████████████████████████████████████████████████████| 433/433 [02:59<00:00,  2.41it/s]


In [156]:
lst_afeautures_flat = []
for lst in lst_afeautures:
    for track in lst['audio_features']:
        lst_afeautures_flat.append(track)

Parece que para algunas canciones no hay información disponible

In [157]:
index_none = []
for index,value in enumerate(lst_afeautures_flat):
    if value is None:
        index_none.append(index)

In [158]:
index_none[::-1]

[23074, 21972, 19003, 6713, 6508, 4200, 2084]

In [159]:
for index in index_none[::-1]:
    lst_afeautures_flat.pop(index);

In [160]:
df_afeautures = pd.DataFrame(lst_afeautures_flat)

In [162]:
df_afeautures.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.577,0.821,1,-6.069,0,0.0681,0.0458,0.000010,0.2350,0.619,87.513,audio_features,6JwkAGwClwUzcOi0ZBbSLe,spotify:track:6JwkAGwClwUzcOi0ZBbSLe,https://api.spotify.com/v1/tracks/6JwkAGwClwUz...,https://api.spotify.com/v1/audio-analysis/6Jwk...,198927,4
1,0.571,0.691,3,-7.027,0,0.0312,0.0873,0.072100,0.1240,0.617,110.004,audio_features,4PawewnYu8lyLk3DX1RCNu,spotify:track:4PawewnYu8lyLk3DX1RCNu,https://api.spotify.com/v1/tracks/4PawewnYu8ly...,https://api.spotify.com/v1/audio-analysis/4Paw...,262511,4
2,0.650,0.280,4,-7.442,1,0.0305,0.4950,0.000004,0.1060,0.103,119.939,audio_features,3PUMPtOSeXSJsBvK43K96b,spotify:track:3PUMPtOSeXSJsBvK43K96b,https://api.spotify.com/v1/tracks/3PUMPtOSeXSJ...,https://api.spotify.com/v1/audio-analysis/3PUM...,382613,3
3,0.490,0.821,7,-7.711,1,0.0848,0.3480,0.000841,0.1980,0.642,195.128,audio_features,3W3GKw4by2pdPfG8bYQMVG,spotify:track:3W3GKw4by2pdPfG8bYQMVG,https://api.spotify.com/v1/tracks/3W3GKw4by2pd...,https://api.spotify.com/v1/audio-analysis/3W3G...,232076,3
4,0.557,0.458,7,-5.127,1,0.0306,0.0499,0.000059,0.1760,0.262,128.129,audio_features,1VZQYv1EhvZAwKYYdgmcdf,spotify:track:1VZQYv1EhvZAwKYYdgmcdf,https://api.spotify.com/v1/tracks/1VZQYv1EhvZA...,https://api.spotify.com/v1/audio-analysis/1VZQ...,238200,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43288,0.442,0.528,10,-9.269,1,0.0562,0.5650,0.029000,0.0944,0.233,132.932,audio_features,4MNu1bSmyV3S4Y2MJMGrrP,spotify:track:4MNu1bSmyV3S4Y2MJMGrrP,https://api.spotify.com/v1/tracks/4MNu1bSmyV3S...,https://api.spotify.com/v1/audio-analysis/4MNu...,311537,4
43289,0.732,0.654,11,-6.842,0,0.0304,0.4840,0.218000,0.1510,0.390,114.007,audio_features,20RzArTj8t5exuBbnszRLz,spotify:track:20RzArTj8t5exuBbnszRLz,https://api.spotify.com/v1/tracks/20RzArTj8t5e...,https://api.spotify.com/v1/audio-analysis/20Rz...,204410,4
43290,0.640,0.472,5,-12.513,1,0.0278,0.4270,0.561000,0.2080,0.331,94.008,audio_features,2F09YSqsNCGLBGvrvUWvZp,spotify:track:2F09YSqsNCGLBGvrvUWvZp,https://api.spotify.com/v1/tracks/2F09YSqsNCGL...,https://api.spotify.com/v1/audio-analysis/2F09...,224599,4
43291,0.705,0.675,1,-6.726,1,0.0406,0.3660,0.551000,0.0968,0.291,96.539,audio_features,5GkPTn5DAUznvHWbofUKA1,spotify:track:5GkPTn5DAUznvHWbofUKA1,https://api.spotify.com/v1/tracks/5GkPTn5DAUzn...,https://api.spotify.com/v1/audio-analysis/5GkP...,201578,4


Generamos un nuevo DataFrame con la información de las canciones y de sus características ("feautures")

In [163]:
df_tracks_feautures = pd.merge(left=tracks,right=df_afeautures, how='left',left_on='track_id',right_on='id')

In [165]:
# Al realizar el pd.merge() se han generado duplicados, por lo que los eliminamos

df_tracks_feautures = df_tracks_feautures.drop_duplicates().reset_index(drop=True)

In [166]:
df_tracks_feautures.head()

,artist,name,track_id,popularity,href,album,album_id,album_href,release_date,danceability,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Del Water Gap,Ode to a Conversation Stuck in Your Throat,6JwkAGwClwUzcOi0ZBbSLe,61,https://api.spotify.com/v1/tracks/6JwkAGwClwUz...,Ode to a Conversation Stuck in Your Throat,5e8S6KvNbnZBbNzaL2YcLJ,https://api.spotify.com/v1/albums/5e8S6KvNbnZB...,2020-05-01,0.577,...,0.235,0.619,87.513,audio_features,6JwkAGwClwUzcOi0ZBbSLe,spotify:track:6JwkAGwClwUzcOi0ZBbSLe,https://api.spotify.com/v1/tracks/6JwkAGwClwUz...,https://api.spotify.com/v1/audio-analysis/6Jwk...,198927.0,4.0
1,Del Water Gap,Better Than I Know Myself,4PawewnYu8lyLk3DX1RCNu,59,https://api.spotify.com/v1/tracks/4PawewnYu8ly...,Del Water Gap,22ljnmjYzy4TS5tCtaRIUE,https://api.spotify.com/v1/albums/22ljnmjYzy4T...,2021-10-08,0.571,...,0.124,0.617,110.004,audio_features,4PawewnYu8lyLk3DX1RCNu,spotify:track:4PawewnYu8lyLk3DX1RCNu,https://api.spotify.com/v1/tracks/4PawewnYu8ly...,https://api.spotify.com/v1/audio-analysis/4Paw...,262511.0,4.0
2,Del Water Gap,New Song,3PUMPtOSeXSJsBvK43K96b,59,https://api.spotify.com/v1/tracks/3PUMPtOSeXSJ...,Notes from the Archive: Recordings 2011-2016,2li1FMT6riytgLdq4Eu4rb,https://api.spotify.com/v1/albums/2li1FMT6riyt...,2020-12-18,0.650,...,0.106,0.103,119.939,audio_features,3PUMPtOSeXSJsBvK43K96b,spotify:track:3PUMPtOSeXSJsBvK43K96b,https://api.spotify.com/v1/tracks/3PUMPtOSeXSJ...,https://api.spotify.com/v1/audio-analysis/3PUM...,382613.0,3.0
3,Del Water Gap,Perfume,3W3GKw4by2pdPfG8bYQMVG,57,https://api.spotify.com/v1/tracks/3W3GKw4by2pd...,Del Water Gap,22ljnmjYzy4TS5tCtaRIUE,https://api.spotify.com/v1/albums/22ljnmjYzy4T...,2021-10-08,0.490,...,0.198,0.642,195.128,audio_features,3W3GKw4by2pdPfG8bYQMVG,spotify:track:3W3GKw4by2pdPfG8bYQMVG,https://api.spotify.com/v1/tracks/3W3GKw4by2pd...,https://api.spotify.com/v1/audio-analysis/3W3G...,232076.0,3.0
4,Del Water Gap,High Tops,1VZQYv1EhvZAwKYYdgmcdf,57,https://api.spotify.com/v1/tracks/1VZQYv1EhvZA...,High Tops,4NMOX3QV2My3pNgg0hu8hX,https://api.spotify.com/v1/albums/4NMOX3QV2My3...,2018-10-11,0.557,...,0.176,0.262,128.129,audio_features,1VZQYv1EhvZAwKYYdgmcdf,spotify:track:1VZQYv1EhvZAwKYYdgmcdf,https://api.spotify.com/v1/tracks/1VZQYv1EhvZA...,https://api.spotify.com/v1/audio-analysis/1VZQ...,238200.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43341,H2O,Like A Prayer,2RsgwouRIGaDw6pEL6TJfw,34,https://api.spotify.com/v1/tracks/2RsgwouRIGaD...,Go,2ISVcKsMGWu5NpsqiRb8Tj,https://api.spotify.com/v1/albums/2ISVcKsMGWu5...,2001-01-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43342,H2O,Everready,03PW4vF6lthRVcZdnfbn7x,33,https://api.spotify.com/v1/tracks/03PW4vF6lthR...,Thicker Than Water,30Ow3FpKrdLWfZijD6j9h9,https://api.spotify.com/v1/albums/30Ow3FpKrdLW...,1997-10-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43343,H2O,Family Tree,3MRHzHjvLtvaLK4qORjxSj,33,https://api.spotify.com/v1/tracks/3MRHzHjvLtva...,H2O,6SIXL2tKLkV6FBZSzf9agC,https://api.spotify.com/v1/albums/6SIXL2tKLkV6...,1996-12-31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43344,H2O,Skate!,1nyhs2wtBpxIe2tbXDHrwl,29,https://api.spotify.com/v1/tracks/1nyhs2wtBpxI...,Use Your Voice,0yVXiixPwL4uJG9Fvq1t8G,https://api.spotify.com/v1/albums/0yVXiixPwL4u...,2015-10-09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Salvamos el DataFrame

In [184]:
df_tracks_feautures.to_parquet('../data/df_tracks_feautures_raw.parquet',engine='pyarrow')

Observamos que hay canciones que no se han incluido en la consulta. Vamos a hacer una segunda comprobación

In [167]:
df_tracks_feautures[df_tracks_feautures.danceability.isna()]['track_id']

2084     5FvcpP9xqu6wAw56f2ZUbE
4200     4iLBQIgE9PuT41Rf8M9bPw
6508     2HWSWCBHKz7OMLk3WqiAW7
6713     16fS9wW6xebfJj0Zd0RHUb
19003    3qv0kUAXxiJrts0FQc9hkc
21972    0bxnZ5LnCcFLKacpBUUxT5
23074    3tc9Kbro2cpggcOWTRTS5x
43300    1G3rpP4SpzMKx7fKu4NZPV
43301    1xhgpTWQFIDvcuBnwVzxPv
43302    3CUd4oB20LXnoWlzQDxTgy
43303    6BUnPZ1HBVOTg9NWsQ2WQL
43304    1xRhjC7hcO3cxKAoHiWsvc
43305    5E0xmqchfJ8HFXfzp2g1Tl
43306    15BQ7vEDv2LJuh8TxWIhtd
43307    21YxK0klhpfLW8budkJaMF
43308    6ZAwEE4kTdtvyE66iRhFxR
43309    6sFpmdsk4UDMcDWdy4T1Kc
43310    1ojRut8t5RYNmLpJkEdJ3V
43311    6OGIBcJV5qV1FF5uiofnX0
43312    27BATZe36zHNAKfr7by2ob
43313    6WPGAupim73K9XQL4iIefZ
43314    0xGFngTneJV4OzN310i8Qh
43315    5JngFjxkVIvlGSsHgUME7B
43316    2W9dSL2pfdAx1H2MzkYw3t
43317    6gHSLe6d5hGx15NC2vRueI
43318    6bjsuoURbXkslNAeScNwFv
43319    50cZaabiwOmgZMhCzU4QXW
43320    2OWag0JAntRlapX8o5vZRw
43321    6BlJj9zsKVr9BLdBJIwxGY
43322    2gfiny9rjptCprs9EdyZ0U
43323    4s4de4CyPyjhmzTEhuT5bb
43324   

In [170]:
lst_afeautures_2 = get_audio_feautures(df_tracks_feautures[df_tracks_feautures.danceability.isna()]['track_id'],headers=headers)

In [171]:
lst_afeautures_2 = lst_afeautures_2['audio_features']
index_none = []
for index,value in enumerate(lst_afeautures_2):
    if value is None:
        index_none.append(index)

for index in index_none[::-1]:
    lst_afeautures_2.pop(index);
    
df_afeautures_2 = pd.DataFrame(lst_afeautures_2)
df_afeautures_2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.489,0.441,11,-9.694,1,0.0288,0.292000,0.339000,0.1320,0.2220,83.903,audio_features,1G3rpP4SpzMKx7fKu4NZPV,spotify:track:1G3rpP4SpzMKx7fKu4NZPV,https://api.spotify.com/v1/tracks/1G3rpP4SpzMK...,https://api.spotify.com/v1/audio-analysis/1G3r...,273165,4
1,0.555,0.707,2,-6.842,1,0.0538,0.279000,0.910000,0.1500,0.3230,74.995,audio_features,1xhgpTWQFIDvcuBnwVzxPv,spotify:track:1xhgpTWQFIDvcuBnwVzxPv,https://api.spotify.com/v1/tracks/1xhgpTWQFIDv...,https://api.spotify.com/v1/audio-analysis/1xhg...,165545,4
2,0.628,0.663,9,-6.924,1,0.0527,0.084700,0.853000,0.5830,0.5710,137.883,audio_features,3CUd4oB20LXnoWlzQDxTgy,spotify:track:3CUd4oB20LXnoWlzQDxTgy,https://api.spotify.com/v1/tracks/3CUd4oB20LXn...,https://api.spotify.com/v1/audio-analysis/3CUd...,165656,4
3,0.295,0.657,11,-7.640,1,0.0384,0.061500,0.869000,0.5090,0.2730,164.139,audio_features,6BUnPZ1HBVOTg9NWsQ2WQL,spotify:track:6BUnPZ1HBVOTg9NWsQ2WQL,https://api.spotify.com/v1/tracks/6BUnPZ1HBVOT...,https://api.spotify.com/v1/audio-analysis/6BUn...,201055,4
4,0.609,0.614,6,-9.907,0,0.0792,0.187000,0.146000,0.3230,0.7710,106.078,audio_features,1xRhjC7hcO3cxKAoHiWsvc,spotify:track:1xRhjC7hcO3cxKAoHiWsvc,https://api.spotify.com/v1/tracks/1xRhjC7hcO3c...,https://api.spotify.com/v1/audio-analysis/1xRh...,208869,4
5,0.606,0.683,5,-6.874,0,0.0462,0.014700,0.943000,0.1020,0.6380,121.874,audio_features,5E0xmqchfJ8HFXfzp2g1Tl,spotify:track:5E0xmqchfJ8HFXfzp2g1Tl,https://api.spotify.com/v1/tracks/5E0xmqchfJ8H...,https://api.spotify.com/v1/audio-analysis/5E0x...,181843,4
6,0.498,0.889,0,-5.413,1,0.0546,0.094700,0.000002,0.7330,0.6960,134.880,audio_features,15BQ7vEDv2LJuh8TxWIhtd,spotify:track:15BQ7vEDv2LJuh8TxWIhtd,https://api.spotify.com/v1/tracks/15BQ7vEDv2LJ...,https://api.spotify.com/v1/audio-analysis/15BQ...,283360,4
7,0.383,0.970,9,-5.638,1,0.0435,0.027600,0.000000,0.0942,0.6200,101.603,audio_features,21YxK0klhpfLW8budkJaMF,spotify:track:21YxK0klhpfLW8budkJaMF,https://api.spotify.com/v1/tracks/21YxK0klhpfL...,https://api.spotify.com/v1/audio-analysis/21Yx...,313926,4
8,0.590,0.851,4,-11.754,1,0.0297,0.017400,0.097500,0.1540,0.9680,146.213,audio_features,6ZAwEE4kTdtvyE66iRhFxR,spotify:track:6ZAwEE4kTdtvyE66iRhFxR,https://api.spotify.com/v1/tracks/6ZAwEE4kTdtv...,https://api.spotify.com/v1/audio-analysis/6ZAw...,218307,4
9,0.624,0.585,7,-10.812,1,0.0300,0.041400,0.000021,0.0467,0.6460,144.251,audio_features,6sFpmdsk4UDMcDWdy4T1Kc,spotify:track:6sFpmdsk4UDMcDWdy4T1Kc,https://api.spotify.com/v1/tracks/6sFpmdsk4UDM...,https://api.spotify.com/v1/audio-analysis/6sFp...,253493,4


In [199]:
index_nan = df_tracks_feautures[df_tracks_feautures.track_id.isin(df_afeautures_2.id)].index
df_tracks_feautures.iloc[index_nan,9:] =df_afeautures_2

Eliminamos duplicados

In [200]:
df_tracks_feautures[df_tracks_feautures.danceability.isna()]

,artist,name,track_id,popularity,href,album,album_id,album_href,release_date,danceability,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
2084,Sam Paganini,Polyester,5FvcpP9xqu6wAw56f2ZUbE,32,https://api.spotify.com/v1/tracks/5FvcpP9xqu6w...,Polyester,5y7JlSx3o2nJWS0UCXziCZ,https://api.spotify.com/v1/albums/5y7JlSx3o2nJ...,2021-05-14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4200,Banks Arcade,Be Someone,4iLBQIgE9PuT41Rf8M9bPw,38,https://api.spotify.com/v1/tracks/4iLBQIgE9PuT...,Future Lovers,4i8gvZZJAG88UAG4Jy7vEl,https://api.spotify.com/v1/albums/4i8gvZZJAG88...,2022-07-29,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6508,Lucifer,Feel Good Inc,2HWSWCBHKz7OMLk3WqiAW7,45,https://api.spotify.com/v1/tracks/2HWSWCBHKz7O...,Feel Good Inc,6hfalf8AUELQjMjkKD8RQm,https://api.spotify.com/v1/albums/6hfalf8AUELQ...,2021-05-21,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6713,Midnight,Golden Touch,16fS9wW6xebfJj0Zd0RHUb,38,https://api.spotify.com/v1/tracks/16fS9wW6xebf...,Touch,62odbR4Mss645oRxdI4nHs,https://api.spotify.com/v1/albums/62odbR4Mss64...,2023-04-15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19003,nasTia,Disco Breakaway,3qv0kUAXxiJrts0FQc9hkc,0,https://api.spotify.com/v1/tracks/3qv0kUAXxiJr...,Disco Breakaway,3imm0qw2BA1JFWlSPcUfHu,https://api.spotify.com/v1/albums/3imm0qw2BA1J...,2021-06-27,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21972,Wallowing,Obliteration,0bxnZ5LnCcFLKacpBUUxT5,15,https://api.spotify.com/v1/tracks/0bxnZ5LnCcFL...,Earth Reaper,2wYklGu9ZIUtTJDWudvbVS,https://api.spotify.com/v1/albums/2wYklGu9ZIUt...,2023-04-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23074,Miss K8,Bogota - 2020 Refix,3tc9Kbro2cpggcOWTRTS5x,46,https://api.spotify.com/v1/tracks/3tc9Kbro2cpg...,Immortality EP,1fGsNOh8B2mIN9KQ8Jt7gh,https://api.spotify.com/v1/albums/1fGsNOh8B2mI...,2020-05-08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
df_tracks_feautures = df_tracks_feautures.dropna().reset_index(drop = True)

In [210]:
df_tracks_feautures.head()

,artist,name,track_id,popularity,href,album,album_id,album_href,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Del Water Gap,Ode to a Conversation Stuck in Your Throat,6JwkAGwClwUzcOi0ZBbSLe,61,https://api.spotify.com/v1/tracks/6JwkAGwClwUz...,Ode to a Conversation Stuck in Your Throat,5e8S6KvNbnZBbNzaL2YcLJ,https://api.spotify.com/v1/albums/5e8S6KvNbnZB...,2020-05-01,0.577,0.821,1.0,-6.069,0.0,0.0681,0.0458,0.000010,0.235,0.619,87.513,audio_features,6JwkAGwClwUzcOi0ZBbSLe,spotify:track:6JwkAGwClwUzcOi0ZBbSLe,https://api.spotify.com/v1/tracks/6JwkAGwClwUz...,https://api.spotify.com/v1/audio-analysis/6Jwk...,198927.0,4.0
1,Del Water Gap,Better Than I Know Myself,4PawewnYu8lyLk3DX1RCNu,59,https://api.spotify.com/v1/tracks/4PawewnYu8ly...,Del Water Gap,22ljnmjYzy4TS5tCtaRIUE,https://api.spotify.com/v1/albums/22ljnmjYzy4T...,2021-10-08,0.571,0.691,3.0,-7.027,0.0,0.0312,0.0873,0.072100,0.124,0.617,110.004,audio_features,4PawewnYu8lyLk3DX1RCNu,spotify:track:4PawewnYu8lyLk3DX1RCNu,https://api.spotify.com/v1/tracks/4PawewnYu8ly...,https://api.spotify.com/v1/audio-analysis/4Paw...,262511.0,4.0
2,Del Water Gap,New Song,3PUMPtOSeXSJsBvK43K96b,59,https://api.spotify.com/v1/tracks/3PUMPtOSeXSJ...,Notes from the Archive: Recordings 2011-2016,2li1FMT6riytgLdq4Eu4rb,https://api.spotify.com/v1/albums/2li1FMT6riyt...,2020-12-18,0.650,0.280,4.0,-7.442,1.0,0.0305,0.4950,0.000004,0.106,0.103,119.939,audio_features,3PUMPtOSeXSJsBvK43K96b,spotify:track:3PUMPtOSeXSJsBvK43K96b,https://api.spotify.com/v1/tracks/3PUMPtOSeXSJ...,https://api.spotify.com/v1/audio-analysis/3PUM...,382613.0,3.0
3,Del Water Gap,Perfume,3W3GKw4by2pdPfG8bYQMVG,57,https://api.spotify.com/v1/tracks/3W3GKw4by2pd...,Del Water Gap,22ljnmjYzy4TS5tCtaRIUE,https://api.spotify.com/v1/albums/22ljnmjYzy4T...,2021-10-08,0.490,0.821,7.0,-7.711,1.0,0.0848,0.3480,0.000841,0.198,0.642,195.128,audio_features,3W3GKw4by2pdPfG8bYQMVG,spotify:track:3W3GKw4by2pdPfG8bYQMVG,https://api.spotify.com/v1/tracks/3W3GKw4by2pd...,https://api.spotify.com/v1/audio-analysis/3W3G...,232076.0,3.0
4,Del Water Gap,High Tops,1VZQYv1EhvZAwKYYdgmcdf,57,https://api.spotify.com/v1/tracks/1VZQYv1EhvZA...,High Tops,4NMOX3QV2My3pNgg0hu8hX,https://api.spotify.com/v1/albums/4NMOX3QV2My3...,2018-10-11,0.557,0.458,7.0,-5.127,1.0,0.0306,0.0499,0.000059,0.176,0.262,128.129,audio_features,1VZQYv1EhvZAwKYYdgmcdf,spotify:track:1VZQYv1EhvZAwKYYdgmcdf,https://api.spotify.com/v1/tracks/1VZQYv1EhvZA...,https://api.spotify.com/v1/audio-analysis/1VZQ...,238200.0,4.0


In [212]:
# Nos quedamos con las columnas que nos interesa

df_tracks_feautures = df_tracks_feautures.drop(columns=['type','id','uri','track_href'])

In [213]:
# Salvoamos la información

df_tracks_feautures.to_parquet('../data/df_tracks_feautures_clean.parquet')

<a name='load'/>

###### ⬆️ [Indice](#ind)

### Carga
***

In [214]:
cursor = pymongo.MongoClient('mongodb://localhost:27017')

In [215]:
festivales = cursor.Festivales

In [216]:
track_feature = festivales.track_feature_colec

In [217]:
track_feature.insert_many(df_tracks_feautures.to_dict(orient='records'))